<img src=https://raw.githubusercontent.com/hoittoken/Python/master/Py/project_3/logo_2.png>

# Содержание <a class="anchor" id=0></a>

* [1. Загрузка данных](#1)
* [2. Признаки туда - сюда](#2)
* [3. Модель. Метрика.](#3)


### **ЗАДАЧА**: Быстро и неряшливо добиться результата близкого [к моему же решению](https://www.kaggle.com/code/aubakirov/project-3-eda-fe-aubakirov/notebook) . 


# 1. Загрузка <a class="anchor" id=1></a> <a id="1"></a> <br>

[к содержанию](#0)



In [1]:
# Импортируем основные библиотеки

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


# для кодирования
import category_encoders as ce


# для разделения датасета:
from sklearn.model_selection import train_test_split

# инструменты для построения модели:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
RANDOM_SEED = 42

/kaggle/input/sf-booking/hotels_test.csv
/kaggle/input/sf-booking/hotels_train.csv
/kaggle/input/sf-booking/submission.csv


In [2]:
# Подгрузим наши данные из соревнования

DATA_DIR = '/kaggle/input/sf-booking/'
df_train = pd.read_csv(DATA_DIR+'/hotels_train.csv') # датасет для обучения
df_test = pd.read_csv(DATA_DIR+'hotels_test.csv') # датасет для предсказания
sample_submission = pd.read_csv(DATA_DIR+'/submission.csv') # самбмишн

# сразу нахер дубли с трейна
df_train = df_train.drop_duplicates()

# 2. Признаки туда - сюда <a class="anchor" id=2></a> <a id="2"></a> <br>

[к содержанию](#0)


In [3]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['reviewer_score'] = 0 # в тесте у нас нет значения reviewer_score, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем


In [4]:
%%time
# Приведем дату к формату Datetime
data['review_date'] = pd.to_datetime(data['review_date'])
# Возраст отзыва
data['days_from_today'] = (pd.Timestamp('today') - data['review_date']).dt.days
# Выделим год
data['year'] = data['review_date'].dt.year
# Выделим месяц
data['month'] = data['review_date'].dt.month
# День недели
data['day_of_week'] = data['review_date'].dt.dayofweek
# Дней с чего-то такм
data['days_since_review'] = data['days_since_review'].apply(lambda x: x.split(sep=' ')[0]).astype('int64')


# Посчитаем общее количество слов в позитивных и негативных отзывах 
data['review_total_word_counts'] = data['review_total_negative_word_counts'] + data['review_total_positive_word_counts']

# Введём пропорцию количество негативных / количество позитивных слов ( если слов нет установим значение 0.5)
data['review_words_proportion'] = data.apply(
    lambda x: 0.5 if x['review_total_word_counts'] == 0 \
    else (x['review_total_negative_word_counts']+1) / (x['review_total_positive_word_counts']+1), axis=1)

# заменим текст отзывов на 0 или 1, если есть негативные или позитивные
data['negative_review'] = data['negative_review'].apply(lambda x: 0 if x=='No Negative' else 1)
data['positive_review'] = data['positive_review'].apply(lambda x: 0 if x=='No Positive' else 1)

CPU times: user 14.3 s, sys: 328 ms, total: 14.6 s
Wall time: 14.6 s


In [5]:
data = data.drop([
    'hotel_address','review_date','hotel_name','reviewer_nationality','tags','lat','lng'
], axis=1)

In [6]:
%%time
# Проверим на пропуски и бесконечности
data.isin([np.inf, -np.inf, np.nan]).any().any()

CPU times: user 92.2 ms, sys: 67.1 ms, total: 159 ms
Wall time: 159 ms


False

In [7]:
data.columns

Index(['additional_number_of_scoring', 'average_score', 'negative_review',
       'review_total_negative_word_counts', 'total_number_of_reviews',
       'positive_review', 'review_total_positive_word_counts',
       'total_number_of_reviews_reviewer_has_given', 'days_since_review',
       'sample', 'reviewer_score', 'days_from_today', 'year', 'month',
       'day_of_week', 'review_total_word_counts', 'review_words_proportion'],
      dtype='object')

In [8]:
data['total_number_of_reviews'].value_counts().count()

1142

In [9]:
%%time

bin_encoder = ce.BinaryEncoder(cols=['average_score']) 
type_bin = bin_encoder.fit_transform(data['average_score'])
data = pd.concat([data, type_bin], axis=1)

bin_encoder = ce.BinaryEncoder(cols=['total_number_of_reviews_reviewer_has_given']) 
type_bin = bin_encoder.fit_transform(data['total_number_of_reviews_reviewer_has_given'])
data = pd.concat([data, type_bin], axis=1)

bin_encoder = ce.BinaryEncoder(cols=['additional_number_of_scoring']) 
type_bin = bin_encoder.fit_transform(data['additional_number_of_scoring'])
data = pd.concat([data, type_bin], axis=1)

bin_encoder = ce.BinaryEncoder(cols=['review_total_negative_word_counts']) 
type_bin = bin_encoder.fit_transform(data['review_total_negative_word_counts'])
data = pd.concat([data, type_bin], axis=1)

CPU times: user 3.36 s, sys: 507 ms, total: 3.87 s
Wall time: 3.87 s


In [10]:
%%time
data = pd.get_dummies(data, columns=['year','month','day_of_week'])

CPU times: user 245 ms, sys: 281 ms, total: 526 ms
Wall time: 526 ms


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515431 entries, 0 to 515430
Data columns (total 68 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   additional_number_of_scoring                  515431 non-null  int64  
 1   average_score                                 515431 non-null  float64
 2   negative_review                               515431 non-null  int64  
 3   review_total_negative_word_counts             515431 non-null  int64  
 4   total_number_of_reviews                       515431 non-null  int64  
 5   positive_review                               515431 non-null  int64  
 6   review_total_positive_word_counts             515431 non-null  int64  
 7   total_number_of_reviews_reviewer_has_given    515431 non-null  int64  
 8   days_since_review                             515431 non-null  int64  
 9   sample                                        51

# 3. Модель. Метрика.<a class="anchor" id=3></a> <a id="3"></a> <br>

[к содержанию](#0)


In [12]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample'], axis=1)

y = train_data.reviewer_score.values            # наш таргет
X = train_data.drop(['reviewer_score'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=RANDOM_SEED)

In [13]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [14]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [15]:
%%time
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  3.7min finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.7s


CPU times: user 14min 43s, sys: 2.47 s, total: 14min 46s
Wall time: 3min 46s


[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.7s finished


In [16]:
# Смотрим метрики 

def mean_absolute_percentage_error(y_tr, y_pr):
    """Получение средней абсолютной ошибки"""
    y_tr, y_pr = np.array(y_tr), np.array(y_pr)
    return np.mean(np.abs((y_tr - y_pr) / y_tr)) * 100

print('MAE:', metrics.mean_absolute_error(y_test, y_pred))
print('MAPE:', round(mean_absolute_percentage_error(y_test, y_pred), 2))

MAE: 0.9435652697899338
MAPE: 13.64


<img src=https://raw.githubusercontent.com/hoittoken/Python/master/Py/project_3/mem_1.png>

In [17]:
test_data = test_data.drop(['reviewer_score'], axis=1)
predict_submission = model.predict(test_data)
sample_submission['reviewer_score'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    1.3s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    2.8s finished
